In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import math

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train = pd.read_csv('Music.csv')

In [3]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean()) # выбранная мной метрика

# Важности признаков, определяемые лесом из sklearn

In [144]:
X = train.drop('Rating', 1)
y= train.Rating
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state = 1234)
forest = RandomForestRegressor(max_depth=5)
forest.fit(X_train, y_train)
y_predict = forest.predict(X_test)
rmse(y_test, y_predict)

21.651077492775677

In [145]:
importances = pd.DataFrame({
    'feature': X.columns,
    'importance': np.round(forest.feature_importances_, 3)
})

importances.sort_values("importance", ascending=False)

,feature,importance
1,Track,0.547
0,Artist,0.347
3,Time,0.067
2,User,0.039


In [146]:
forest = RandomForestRegressor(max_depth=5) # модель, подаваемая функции

# Построение функции

In [147]:
def get_importances(model, X, y, metric): # получение важности с шаффлом валидационной выборки
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state = 1234)
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    orig_accur = metric(y_test, y_predict)
    
    X_test = X_test.reset_index(drop=True)
    differences = {}
    features = X.columns.values
    for feat in features:
        X_test2 = X_test.copy()
        np.random.shuffle(X_test2[feat])
        y_predict = model.predict(X_test2)
        new_accur = metric(y_test, y_predict)
        difference = math.fabs(orig_accur - new_accur)
        differences.update({feat: difference})
    
    values = differences.values()
    features = differences.keys()
    summa = sum(values)
    importances = []
    for val in values:
        importances.append(np.round(val/summa, 3))
    
    feat_imp = pd.DataFrame({
    'feature': list(features),
    'importance': importances
    })
    
    return(feat_imp.sort_values("importance", ascending=False))

# Важности признков, определяемые построенной функцией

In [148]:
get_importances(forest, X, y, rmse)

,feature,importance
0,Artist,0.481
2,Track,0.410
3,Time,0.061
1,User,0.048
